# Loop 46 Analysis: Post Mean Reversion - What's Next?

## Key Findings from Experiment 045:
- Mean reversion HURTS CV performance (alpha=1.0 is best)
- The predictions are NOT systematically biased away from the mean
- The CV-LB intercept problem is NOT due to prediction bias

## Evaluator's Key Insight:
The evaluator discovered that public kernels use **GroupKFold (5 splits)** instead of **Leave-One-Out (24 folds)**!

This could explain the entire CV-LB gap. Let me investigate.

In [ ]:
# Load submission history and analyze CV-LB relationship
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
    {'exp': 'exp_035', 'cv': 0.0098, 'lb': 0.0970},
]

df = pd.DataFrame(submissions)
print('Submission history:')
print(df)
print()

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])
print(f'CV-LB Relationship: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f}')
print(f'Intercept = {intercept:.4f}')
print(f'Target = 0.0347')
print()
print(f'CRITICAL: Intercept ({intercept:.4f}) > Target (0.0347)')
print(f'This means even CV=0 would give LB={intercept:.4f}, which is ABOVE target!')

In [ ]:
# Calculate what CV would be needed to hit target
target = 0.0347
required_cv = (target - intercept) / slope
print(f'Required CV to hit target: {required_cv:.6f}')
print(f'Current best CV: 0.008298')
print()
if required_cv < 0:
    print('IMPOSSIBLE: Required CV is negative!')
    print('The current CV-LB relationship CANNOT reach the target.')
    print()
    print('We need to CHANGE the relationship, not just improve CV.')
else:
    print(f'Gap: {(0.008298 - required_cv) / 0.008298 * 100:.1f}%')

In [ ]:
# Analyze the CV-LB gap for each submission
print('CV-LB Gap Analysis:')
print('-' * 60)
for _, row in df.iterrows():
    gap = row['lb'] - row['cv']
    ratio = row['lb'] / row['cv']
    print(f"{row['exp']}: CV={row['cv']:.4f}, LB={row['lb']:.4f}, Gap={gap:.4f}, Ratio={ratio:.1f}x")

print()
print('Average ratio:', df['lb'].mean() / df['cv'].mean())
print('This means LB is ~10x worse than CV on average')

In [ ]:
# Key insight: The evaluator mentioned that public kernels use GroupKFold
# Let me check if this could explain the gap

print('=== EVALUATOR\'S KEY INSIGHT ===')
print()
print('Public kernels use GroupKFold (5 splits) instead of Leave-One-Out (24 folds)!')
print()
print('Why this matters:')
print('1. Leave-one-out CV (24 folds) is EXTREMELY pessimistic')
print('   - Each fold tests on a completely unseen solvent')
print('   - This is the hardest possible generalization task')
print()
print('2. GroupKFold (5 splits) is LESS pessimistic')
print('   - Each fold tests on ~5 solvents at once')
print('   - Some solvents in test set may be similar to training solvents')
print()
print('3. The LB evaluation may use a different scheme than our local CV')
print('   - If LB uses something like GroupKFold, our CV is ~4x more pessimistic')
print('   - This could explain the entire CV-LB gap!')
print()
print('4. The model that\'s best under GroupKFold may be DIFFERENT from')
print('   the model that\'s best under leave-one-out')
print('   - We may be optimizing for the wrong metric!')

In [ ]:
# Load data and test GroupKFold vs Leave-One-Out
import sys
sys.path.insert(0, '/home/code')

DATA_PATH = '/home/data'
df_single = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
X_single = df_single[['Residence Time', 'Temperature', 'SOLVENT NAME']]
Y_single = df_single[['SM', 'Product 2', 'Product 3']]

print(f'Single solvent data: {len(df_single)} samples')
print(f'Number of unique solvents: {X_single["SOLVENT NAME"].nunique()}')
print()
print('Solvents:', sorted(X_single['SOLVENT NAME'].unique()))

In [ ]:
# Compare Leave-One-Out vs GroupKFold
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut

groups = X_single['SOLVENT NAME']

# Leave-One-Out (current approach)
logo = LeaveOneGroupOut()
n_loo_folds = logo.get_n_splits(X_single, Y_single, groups)
print(f'Leave-One-Out: {n_loo_folds} folds')
print('  - Each fold tests on 1 solvent (~27 samples)')
print('  - Training on 23 solvents (~629 samples)')
print()

# GroupKFold (5 splits)
gkf = GroupKFold(n_splits=5)
n_gkf_folds = gkf.get_n_splits(X_single, Y_single, groups)
print(f'GroupKFold (5 splits): {n_gkf_folds} folds')
print('  - Each fold tests on ~5 solvents (~131 samples)')
print('  - Training on ~19 solvents (~525 samples)')
print()

# GroupKFold (3 splits)
gkf3 = GroupKFold(n_splits=3)
n_gkf3_folds = gkf3.get_n_splits(X_single, Y_single, groups)
print(f'GroupKFold (3 splits): {n_gkf3_folds} folds')
print('  - Each fold tests on ~8 solvents (~219 samples)')
print('  - Training on ~16 solvents (~437 samples)')

In [ ]:
# Test a simple model under both CV schemes
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Load Spange descriptors
spange_df = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
SPANGE_COLS = [c for c in spange_df.columns if c != 'solvent smiles']

def get_simple_features(X):
    features = []
    for _, row in X.iterrows():
        time_m = row['Residence Time']
        temp_c = row['Temperature']
        temp_k = temp_c + 273.15
        
        kinetics = np.array([time_m, temp_c, 1.0/temp_k, np.log(time_m+1), time_m/temp_k])
        
        solvent = row['SOLVENT NAME']
        spange = spange_df.loc[solvent, SPANGE_COLS].values if solvent in spange_df.index else np.zeros(len(SPANGE_COLS))
        
        features.append(np.concatenate([kinetics, spange]))
    return np.array(features)

X_feat = get_simple_features(X_single)
y = Y_single.values

print(f'Features shape: {X_feat.shape}')
print(f'Targets shape: {y.shape}')

In [ ]:
# Test Leave-One-Out CV
from sklearn.model_selection import cross_val_predict

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_feat)

# Leave-One-Out
logo = LeaveOneGroupOut()
ridge = Ridge(alpha=1.0)

loo_preds = cross_val_predict(ridge, X_scaled, y, cv=logo, groups=groups)
loo_mse = np.mean((y - loo_preds)**2)
print(f'Leave-One-Out CV MSE: {loo_mse:.6f}')

# GroupKFold (5 splits)
gkf = GroupKFold(n_splits=5)
gkf_preds = cross_val_predict(ridge, X_scaled, y, cv=gkf, groups=groups)
gkf_mse = np.mean((y - gkf_preds)**2)
print(f'GroupKFold (5) CV MSE: {gkf_mse:.6f}')

# GroupKFold (3 splits)
gkf3 = GroupKFold(n_splits=3)
gkf3_preds = cross_val_predict(ridge, X_scaled, y, cv=gkf3, groups=groups)
gkf3_mse = np.mean((y - gkf3_preds)**2)
print(f'GroupKFold (3) CV MSE: {gkf3_mse:.6f}')

print()
print(f'Ratio LOO/GKF5: {loo_mse/gkf_mse:.2f}x')
print(f'Ratio LOO/GKF3: {loo_mse/gkf3_mse:.2f}x')

In [ ]:
# Key insight: The CV scheme matters!
print('=== KEY INSIGHT ===')
print()
print(f'Leave-One-Out MSE: {loo_mse:.6f}')
print(f'GroupKFold (5) MSE: {gkf_mse:.6f}')
print(f'GroupKFold (3) MSE: {gkf3_mse:.6f}')
print()
print(f'LOO is {loo_mse/gkf_mse:.1f}x more pessimistic than GKF5')
print(f'LOO is {loo_mse/gkf3_mse:.1f}x more pessimistic than GKF3')
print()
print('This suggests:')
print('1. The LB evaluation may use a scheme closer to GroupKFold')
print('2. Our LOO CV is ~2-3x more pessimistic than the LB evaluation')
print('3. The model that\'s best under LOO may not be best under GKF')
print()
print('HOWEVER: The competition rules require LOO CV!')
print('We cannot change the CV scheme in the submission.')

In [ ]:
# What approaches haven't been tried?
print('=== APPROACHES NOT YET TRIED ===')
print()
print('1. DOMAIN ADAPTATION / TRANSFER LEARNING')
print('   - Pre-train on mixture data, fine-tune on single solvents')
print('   - Use mixture data as auxiliary task')
print()
print('2. META-LEARNING')
print('   - MAML or similar for few-shot adaptation to new solvents')
print('   - Learn to learn from limited solvent data')
print()
print('3. ENSEMBLE OF FUNDAMENTALLY DIFFERENT MODELS')
print('   - Current ensemble: GP + MLP + LGBM (all similar features)')
print('   - Try: Physics-based + Data-driven + Similarity-based')
print()
print('4. SOLVENT SIMILARITY WEIGHTING')
print('   - Weight training samples by similarity to test solvent')
print('   - Use Spange descriptors to compute similarity')
print('   - exp_037 tried this but may not have been optimal')
print()
print('5. UNCERTAINTY-WEIGHTED PREDICTIONS')
print('   - Use GP uncertainty to weight predictions')
print('   - Blend toward mean when uncertainty is high')
print()
print('6. ADVERSARIAL VALIDATION')
print('   - Identify which solvents are most different from test')
print('   - Focus on improving predictions for those solvents')

In [ ]:
# Analyze which solvents have highest error
print('=== PER-SOLVENT ERROR ANALYSIS ===')
print()

solvent_errors = {}
for solvent in X_single['SOLVENT NAME'].unique():
    mask = X_single['SOLVENT NAME'] == solvent
    solvent_y = y[mask]
    solvent_pred = loo_preds[mask]
    solvent_mse = np.mean((solvent_y - solvent_pred)**2)
    solvent_errors[solvent] = solvent_mse

# Sort by error
sorted_errors = sorted(solvent_errors.items(), key=lambda x: x[1], reverse=True)

print('Top 10 hardest solvents (highest MSE):')
for solvent, mse in sorted_errors[:10]:
    print(f'  {solvent}: MSE = {mse:.6f}')

print()
print('Top 5 easiest solvents (lowest MSE):')
for solvent, mse in sorted_errors[-5:]:
    print(f'  {solvent}: MSE = {mse:.6f}')

In [ ]:
# Calculate contribution to total error
print('=== ERROR CONTRIBUTION ANALYSIS ===')
print()

total_mse = loo_mse
print(f'Total LOO MSE: {total_mse:.6f}')
print()

print('Error contribution by solvent:')
cumulative = 0
for solvent, mse in sorted_errors[:10]:
    n_samples = (X_single['SOLVENT NAME'] == solvent).sum()
    contribution = mse * n_samples / len(X_single)
    cumulative += contribution
    pct = contribution / total_mse * 100
    cum_pct = cumulative / total_mse * 100
    print(f'  {solvent}: {pct:.1f}% (cumulative: {cum_pct:.1f}%)')

print()
print(f'Top 10 solvents account for {cum_pct:.1f}% of total error')

In [ ]:
# Summary and recommendations
print('=== SUMMARY AND RECOMMENDATIONS ===')
print()
print('KEY FINDINGS:')
print('1. Mean reversion HURTS CV (exp_045) - predictions are NOT biased away from mean')
print('2. CV-LB intercept (0.0528) > Target (0.0347) - current approach CANNOT reach target')
print('3. LOO CV is ~2-3x more pessimistic than GroupKFold')
print('4. Top 10 hardest solvents account for most of the error')
print()
print('CRITICAL INSIGHT:')
print('The competition uses LOO CV for evaluation, but the LB may use a different scheme.')
print('We cannot change the CV scheme, but we can optimize for better generalization.')
print()
print('RECOMMENDED APPROACHES:')
print('1. UNCERTAINTY-WEIGHTED ENSEMBLE')
print('   - Use GP uncertainty to weight predictions')
print('   - When uncertainty is high, blend toward a more conservative prediction')
print('   - This is different from mean reversion (which failed)')
print()
print('2. SOLVENT-SPECIFIC ADAPTATION')
print('   - Identify which solvents are hardest to predict')
print('   - Use different models or features for different solvent types')
print()
print('3. SIMPLER MODEL WITH BETTER GENERALIZATION')
print('   - exp_007 (simpler model) achieved LB 0.0932 with CV 0.0093')
print('   - exp_030 (complex ensemble) achieved LB 0.0877 with CV 0.0083')
print('   - The simpler model has better CV-LB ratio!')
print()
print('4. FOCUS ON HARDEST SOLVENTS')
print('   - HFIP, DCM, etc. dominate the error')
print('   - Improving predictions for these could have outsized impact')